# Gremlin -  Ejemplo Videojuegos

<img src="http://neptune.oldschool.cloud/images/graph.jpg" width="300" height="300">

## Introduccion

En este cuaderno vamos a examinar algunas de las consultas Gremlin para explorar las propiedades de nuestro Grafo. Como hemos visto, hay Vertices (V) para Personas y para Videojuegos. Hay Bordes (E) entre las personas y los juegos que definen que Videojuego le gusta a la Persona y en que grado.



### Primeramente revisemos el estado del cluster de Neptune
Antes de ejecutar alguna consulta, vamos a asegurarnos que estamos conectados a Neptune y que el clustet esta listo. Notar que el comando `%graph_notebook_config` regresara la configuracion actual del cuaderno de python.

In [66]:
%graph_notebook_version

1.27


In [67]:
%graph_notebook_config

{
  "host": "neptunedbcluster-pxjosopjfuru.cluster-cgcclj1nemxy.us-east-1.neptune.amazonaws.com",
  "port": 8182,
  "auth_mode": "DEFAULT",
  "iam_credentials_provider_type": "ROLE",
  "load_from_s3_arn": "",
  "ssl": true,
  "aws_region": "us-east-1"
}


In [39]:
%status

{'status': 'healthy',
 'startTime': 'Fri Aug 07 02:35:45 UTC 2020',
 'dbEngineVersion': '1.0.3.0.R1',
 'role': 'writer',
 'gremlin': {'version': 'tinkerpop-3.4.3'},
 'sparql': {'version': 'sparql-1.1'},
 'labMode': {'ObjectIndex': 'disabled',
  'DFEQueryEngine': 'disabled',
  'ReadWriteConflictDetection': 'enabled'}}

## Representacion Visual de una Consulta

Los resultados de calquier consulta de Gremlin que regrese un `path` puede ser exploara visualmente. Cuando esas consultas se ejecutan, podras ver una pestana nueva llamada `Graph` en el area de resultados de las consultas junto a la pestana `Console`.


## Ejemplos de consultas Gremlin
Las siguientes celdas contien algunas de las consultas que usamos desde la la linea de comando en los pasos anteriores del laboratorio, ademas de algunas consultas adicionales que nos permitiran explorar visuzalizaciones.

### Consulta para un vértice particular (jugador):

In [72]:
%%gremlin
g.V().hasId('Luke').valueMap()

In [73]:
%%gremlin
g.V().has("GamerAlias","skywalker123").valueMap()


In [74]:
%%gremlin
g.V().has('GamerAlias','skywalker123')


### Muestra algunos de los bordes (límite 5):

In [75]:
%%gremlin
g.E().limit(5)


### Muestra algunos de los vértices (límite 4):

In [76]:
%%gremlin
g.V().limit(4)


### Cuente la centralidad en grados de los bordes entrantes para cada vértice:

In [77]:
%%gremlin
g.V().group().by().by(inE().count()).unfold()


### Cuente la centralidad de los bordes salientes de cada vértice:

In [78]:
%%gremlin
g.V().group().by().by(outE().count()).unfold()


### Cuente la centralidad del grado de salida de los bordes salientes de cada vértice por orden de grado:

In [79]:
%%gremlin
g.V().project("v","degree").by().by(bothE().count()).order().by(select("degree"), decr)


### Devuelve solo los vértices que son juegos:

In [119]:
%%gremlin
g.V().hasLabel('game')



 ### Devuelve solo los vértices que son jugadores:


In [85]:
%%gremlin
g.V().hasLabel('person')



### Recuento de juegos agrupados por género de juego:


In [86]:
%%gremlin
g.V().hasLabel('game').groupCount().by("GameGenre").unfold()


### Recuento de juegos agrupados por desarrollador:

In [89]:
%%gremlin
g.V().hasLabel('game').groupCount().by("Developer").unfold()


### Recuento de juegos agrupados por plataforma:


In [90]:
%%gremlin
g.V().hasLabel('game').groupCount().by("Platform").unfold()


### ¿Cuál es la calificación promedio ponderada de MarioKart8?


In [91]:
%%gremlin
g.V().hasLabel('game').has('GameTitle','MarioKart8').inE('likes').values('weight').mean()


### Qué juegos le gustan a skywalker123?

In [131]:
%%gremlin
g.V().has('GamerAlias','skywalker123').as('gamer').out('likes')


### ¿Qué juegos le gusta a skywalker123 usar el peso (mayor que)?

In [132]:
%%gremlin
g.V().has('GamerAlias','skywalker123').outE("likes").has('weight', P.gt(0.7f))


### ¿Qué juegos le gusta a skywalker123 usar el peso (menos que)?

In [133]:
%%gremlin
g.V().has('GamerAlias','skywalker123').outE("likes").has('weight', P.lt(0.5f))


### ¿A quién más le gustan los mismos juegos que al jugador skywalker123?

In [134]:
%%gremlin
g.V().has('GamerAlias','skywalker123').out('likes').in('likes').dedup().values('GamerAlias')


### ¿A quién más le gustan estos juegos sin incluirse a uno mismo (skywalker123)?

In [135]:
%%gremlin
g.V().has('GamerAlias','skywalker123').as('TargetGamer').out('likes').in('likes').where(neq('TargetGamer')).dedup().values('GamerAlias')


### ¿Cuáles son los otros títulos de juegos que les gustan a otros jugadores, que tienen en común?

In [136]:
%%gremlin
g.V().has('GamerAlias','skywalker123').as('TargetGamer').out('likes').in('likes').where(neq('TargetGamer')).out('likes').dedup().values('GameTitle')


### ¿Qué juegos podrían tener sentido recomendar a un jugador específico que actualmente no le gusta?

In [137]:
%%gremlin
g.V().has('GamerAlias','skywalker123').as('TargetGamer').out('likes').aggregate('self').in('likes').where(neq('TargetGamer')).out('likes').where(without('self')).dedup().values('GameTitle')


### Vamos a revisar ahora Vertices y los Bordes en el Grafo.
Cada vertice tiene una etiqueta que escencialmente le da un tipo.

In [138]:
%%gremlin
g.V().groupCount().by(label)

De la misma forma, cada Borde (Edge) tiene una etiqueta.

In [139]:
%%gremlin
g.E().groupCount().by(label)

## Visualizaciones
Ejecute la siguiente consulta, y haga clic en la pestana del Grafo para ver la visualizacion. Solo las consultas que regresen un path podran generar una visuaalizacion.

### Visualizar todos los Vertices que tienen la etiqueta 'person' con sus Bordes de salida (outE)

In [164]:
%%gremlin -p v,oute,inv
g.V().hasLabel('person').outE().inV().path()


### Visualizar los Juegos que le giustan al jugador 'skywalker' 

In [163]:
%%gremlin -p v,oute,inv
g.V().has('GamerAlias','skywalker123').outE("likes").inV().path()

### Visualizar los Juegos que le giustan al jugador 'forchinet' 

In [162]:
%%gremlin -p v,oute,inv
g.V().has('GamerAlias','forchinet').outE("likes").inV().path()

## Cambiar la configuracion de parametros
Los cuadernos de Amazon Neptune usan una libreria de codigo abierto llamada [Vis.js](https://github.com/visjs) para generar los diagramas del grafo. Vis.js provee una gran cantidad de opciones de configuracion. LA documentacion para la mayoria de los ajustes de visualizacion usados en este libroro puedne encontrarse [aqui](https://visjs.org/) y en particular la documentacion de losdiagramas de red del grafo puedne ser encontrados [aqui](https://visjs.github.io/vis-network/docs/network/).

Para ver los parametros actuales usados por el cuaderno puede usar el comando: `%graph_notebook_vis_options` 

In [158]:
%graph_notebook_vis_options

{
  "nodes": {
    "borderWidthSelected": 0,
    "borderWidth": 0,
    "color": {
      "background": "rgba(210, 229, 255, 1)",
      "border": "transparent",
      "highlight": {
        "background": "rgba(9, 104, 178, 1)",
        "border": "rgba(8, 62, 100, 1)"
      }
    },
    "shadow": {
      "enabled": false
    },
    "shape": "circle",
    "widthConstraint": {
      "minimum": 70,
      "maximum": 70
    },
    "font": {
      "face": "courier new",
      "color": "black",
      "size": 12
    }
  },
  "edges": {
    "color": {
      "inherit": false
    },
    "smooth": {
      "enabled": true,
      "type": "straightCross"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "type": "arrow"
      }
    },
    "font": {
      "face": "courier new"
    }
  },
  "interaction": {
    "hover": true,
    "hoverConnectedEdges": true,
    "selectConnectedEdges": false
  },
  "physics": {
    "hierarchicalRepulsion": {
      "centralGravity": 0
    },
    "minVelo

### Producir un diagrama jerarquico
Para algunos tipos de consulta una vista jerarquica es mas adecuada. Ejecute la siguiente celda para cambiar algunos parametros y despues ejecute la siguiente celda para demosrtrar los cambios. Hay una celda despues que puede usar para regresar a valores iniciales.

In [159]:
%%graph_notebook_vis_options
{
  "nodes": {
    "borderWidthSelected": 0,
    "borderWidth": 0,
    "color": {
      "background": "rgba(210, 229, 255, 1)",
      "border": "transparent",
      "highlight": {
        "background": "rgba(9, 104, 178, 1)",
        "border": "rgba(8, 62, 100, 1)"
      }
    },
    "shadow": {
      "enabled": false
    },
    "shape": "circle",
    "widthConstraint": {
      "minimum": 70,
      "maximum": 70
    },
    "font": {
      "face": "courier new",
      "color": "black",
      "size": 12
    }
  },
  "edges": {
    "color": {
      "inherit": false
    },
    "smooth": {
      "enabled": true,
      "type": "straightCross"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "type": "arrow"
      }
    },
    "font": {
      "face": "courier new"
    }
  },
  "interaction": {
    "hover": true,
    "hoverConnectedEdges": true,
    "selectConnectedEdges": false
  },
  "physics": {
    "hierarchicalRepulsion": {
      "centralGravity": 0
    },
    "minVelocity": 0.75,
    "solver": "hierarchicalRepulsion"
  },
  "layout": {
    "hierarchical": { 
      "enabled": true,
      "direction": "LR",
      "sortMethod": "directed",
      "edgeMinimization":false }
  }
}


### Regresar a los valores por defecto
Al ejecutar la celda siguiente, se configuraran los valores por defecto. Sin embargo tambien se puede usar el comando: `%graph_notebook_vis_options reset` y tendras el mismo resultado.

In [160]:
%%graph_notebook_vis_options
{
  "nodes": {
    "borderWidthSelected": 0,
    "borderWidth": 0,
    "color": {
      "background": "rgba(210, 229, 255, 1)",
      "border": "transparent",
      "highlight": {
        "background": "rgba(9, 104, 178, 1)",
        "border": "rgba(8, 62, 100, 1)"
      }
    },
    "shadow": {
      "enabled": false
    },
    "shape": "circle",
    "widthConstraint": {
      "minimum": 70,
      "maximum": 70
    },
    "font": {
      "face": "courier new",
      "color": "black",
      "size": 12
    }
  },
  "edges": {
    "color": {
      "inherit": false
    },
    "smooth": {
      "enabled": true,
      "type": "straightCross"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "type": "arrow"
      }
    },
    "font": {
      "face": "courier new"
    }
  },
  "interaction": {
    "hover": true,
    "hoverConnectedEdges": true,
    "selectConnectedEdges": false
  },
  "physics": {
    "minVelocity": 0.75,
    "barnesHut": {
      "centralGravity": 0.1,
      "gravitationalConstant": -50450,
      "springLength": 95,
      "springConstant": 0.04,
      "damping": 0.09,
      "avoidOverlap": 0.1
    },
    "solver": "barnesHut",
    "enabled": true,
    "adaptiveTimestep": true,
    "stabilization": {
      "enabled": true,
      "iterations": 1
    }
  }
}


### Ver mas datos
Si corres la celda siguiente, el cuaderno usara el maximo de espacio de la ventana del navegador.

In [161]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))